In [ ]:
import torch.nn as nn
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
import torch.utils.data as data

import torch
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
from torchsummary import summary
import os
import cv2

import random
import math
import copy
import time
import gc

In [ ]:
# FRGCv2
num_class = 557  # depth: 557
model_path = 'E:\\LED3D\\newData_training\\Resnet101\\bestModel.pth'  # Resnet101_focal
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# Resnet101_focal
model = models.resnet101().to(device)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_class).to(device)
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.eval()

In [ ]:
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
import os
import csv

image_id = []
image_label = []
label_map = "E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\label_map.csv"

# 開啟 CSV 檔案 讀取label
with open(label_map, newline='',encoding="utf-8-sig") as csvfile:
    
    rows = csv.reader(csvfile)  
    for row in rows:
        image_id.append(row[0])
        image_label.append(row[1])
    
class IMAGE_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.x = []
        self.y = []
        self.transform = transform
        self.num_classes = 0
        
        for i, _dir in enumerate(self.root_dir.glob('*')):
            tmp_str = str(os.path.dirname(_dir))+"\\"   #windows用的是"\",linux用的是"/"這一點要特別清楚，還有就是在寫路徑時用"\\"才表示一個"\"。
            tmp_str = str(_dir).replace(tmp_str, "") 
            for file in _dir.glob('*'):
                self.x.append(file)
                self.y.append(int(image_id.index(tmp_str)))  # 檔名代表label - > 對應的index label
            self.num_classes += 1
            
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        image = Image.open(self.x[index]).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, self.y[index], str(self.x[index])  # img, label, path

整個資料集測試

In [ ]:
from skimage import io
from skimage.metrics import structural_similarity as ssim

# eval
# 改成所需的class
num_class = 557  # depth: 557
model_path = 'E:\\LED3D\\newData_training\\Resnet101\\bestModel.pth'  # Resnet101_focal
batch_size = 64

# Resnet101_focal
model = models.resnet101().to(device)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_class).to(device)
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.eval()
      
mean=[0.5, 0.5, 0.5]
std=[0.5, 0.5, 0.5]

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
depth_path = ['F:\\PointCloud_Attack\\adv_data\\depth_adv_mean=0_std=5']
normal_path = ['F:\\PointCloud_Attack\\adv_data\\normal_adv_mean=0_std=5']
# attack = ['fgsm', 'bim', 'pgd']

for idx in depth_path:
    Depth_dir = os.path.join(idx)
    Depth_ds = IMAGE_Dataset(Depth_dir,trans)
    
    Depth_dataloaders = DataLoader(
        Depth_ds, 
        batch_size= batch_size, 
        pin_memory=False,  
        shuffle=False)

    correct = 0
    total = 0
    ssim_score = 0
    wrong = []

    for images, labels, paths in tqdm(Depth_dataloaders):
        
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, preds = torch.max(outputs, 1)
        
        total += int(images.shape[0])
        correct += (preds == labels).sum()
        
        for t in range(int(images.shape[0])):
            label = os.path.basename(paths[t]).split('d')[0]
            name = os.path.basename(paths[t]).split('_depth')[0]
            img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Depth', label, name)
            image1 = io.imread(paths[t], as_gray=True)  # [256,256,3]
            image2 = io.imread(img_path + '_nu.png', as_gray=True)  # [256,256,3]
            ssim_score += ssim(image1, image2, multichannel=False)
        
    print('This is Depth image ' + ' attack test:')
    print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
    print('number of correct: ', float(correct))
    print('ssim_value = {}%'.format(round((ssim_score*100/total), 5)))
    print('total: ', int(total))

        
torch.cuda.empty_cache()
print('-------------------------------------')

for idx in normal_path: 
    Normal_dir = os.path.join(idx)
    Normal_ds = IMAGE_Dataset(Normal_dir,trans)

    Normal_dataloaders = DataLoader(
        Normal_ds, 
        batch_size= batch_size, 
        pin_memory=False,  
        shuffle=False)

    correct = 0
    total = 0
    ssim_score = 0
    wrong = []

    for images, labels, paths in tqdm(Normal_dataloaders):
        
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, preds = torch.max(outputs, 1)
        
        total += int(images.shape[0])
        correct += (preds == labels).sum()
        
        
        for t in range(int(images.shape[0])):
            label = os.path.basename(paths[t]).split('d')[0]
            name = os.path.basename(paths[t]).split('_normal')[0]
            img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Normal', label, name)
            image1 = io.imread(paths[t], as_gray=False)  # [256,256,3]
            image2 = io.imread(img_path + '_nu.png', as_gray=False)  # [256,256,3]
            ssim_score += ssim(image1, image2, multichannel=True)
            
            # 記錄錯誤 csv
            # if preds[t] != labels[t]:
            #     wrong.append([paths[t], preds[t], labels[t]])
            
        torch.cuda.empty_cache()
        
    print('This is Normal image ' +  ' attack test:')    
    print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
    print('number of correct: ', float(correct))
    print('ssim_value = {}%'.format(round((ssim_score*100/total), 5)))
    print('total: ', int(total))

單樣本測試

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from skimage import io
from skimage.metrics import structural_similarity as ssim

# 定義圖像預處理步驟
mean=[0.5, 0.5, 0.5]
std=[0.5, 0.5, 0.5]
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# 讀取圖像
img_path=['04213d241_depth.png', '04213d241_normal.png', 
          '04213d241_heatmap_gausian_z_0-5_depth.png', '04213d241_heatmap_gausian_z_0-5_normal.png']
for path in img_path:
    input_image = Image.open(path).convert('RGB')

    # 預處理圖像
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)
    input_batch = input_batch.to(device)

    # 模型推斷
    with torch.no_grad():
        output = model(input_batch)

    # 獲取分類結果
    _, predicted = torch.max(output, 1)
    print('File Name: ', path)
    print(f"Predicted class: {predicted.item()}")
    print('image_id:', image_id[predicted.item()])
    print('\n')
    
image0 = io.imread(img_path[0], as_gray=True)  # [256,256,1]
image2 = io.imread(img_path[2], as_gray=True)  # [256,256,1]
ssim_score1 = ssim(image0, image2, multichannel=False)

image1 = io.imread(img_path[1], as_gray=False)  # [256,256,3]
image3 = io.imread(img_path[3], as_gray=False)  # [256,256,3]
ssim_score2 = ssim(image1, image3, multichannel=True)
print('depth image ssim:', ssim_score1)
print('normal image ssim:', ssim_score2)